# MAPT - Subhaplotype analyses

## GP2 NBA data release 7

## Project: Exploring MAPT-containing H1 and H2 haplotypes  in Parkinson's Disease across diverse populations 

Version: Python/3.10.12

Last Updated: 14-MAY-2025

Update Description: Added descriptions to the analysis

Gene coordinates for the region of 17q21.31 (containing MAPT) from the UCSC Browser: chr17:42,800,001-46,800,000 (GRCh38/hg38)

Notebook overview: In this notebook we performed analyses looking at the frequency of subhaplotypes in PD cases and controls in MAPT using six tagging SNPs. In this notebook, we specifically looked at the AAC ancestry group but the analysis was repeated on the other ancestries available in GP2 (with the exception of the FIN due to low sample size).

## Description:

* Loading Python librariess, set paths to the GP2 data and defining functions
* Install packages
* Copy the files
* Create a covariate file
* Remove related individuals
* Remove 'non-PD cases and -controls'
* Extract the region of interest
* Prepare file with the SNPs in the subhaplotype
* Extract the SNPs
* Calculate HWE
*  Run the subhaplotype analysis in R
    * Install R and packages (haplo.stats)
    * Run association analysis between the subhaplotypes and PD
* Save output


### 1. Set up things

#### Loading Python libraries and defining functions

In [ ]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')

## GP2 v7.0
## Explicitly define release v7.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/path/to/release/7'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

print('GP2 v7.0')
print(f'Path to GP2 v7.0 Clinical Data @ `GP2_CLINICAL_RELEASE_PATH`: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Metadata @ `GP2_META_RELEASE_PATH`: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Raw Genotype Data @ `GP2_RAW_GENO_PATH`: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v7.0 Imputed Genotype Data @ `GP2_IMPUTED_GENO_PATH`: {GP2_IMPUTED_GENO_PATH}')
print(f'Path to GP2 v7.0 summary statistics: {GP2_SUMSTAT_RELEASE_PATH}')

In [ ]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

In [ ]:
%%capture
%%bash

# Install plink 1.9
cd /home/jupyter/
if test -e /home/jupyter/plink; then

echo "Plink is already installed in /home/jupyter/"
else
echo "Plink is not installed"
cd /home/jupyter

wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

unzip -o plink_linux_x86_64_20190304.zip
mv plink plink1.9

fi

#### Install plink v1.9 and v2.0

In [ ]:
%%capture
%%bash

# Install plink 1.9
cd /home/jupyter/
if test -e /home/jupyter/plink; then

echo "Plink is already installed in /home/jupyter/"
else
echo "Plink is not installed"
cd /home/jupyter

wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

unzip -o plink_linux_x86_64_20190304.zip
mv plink plink1.9

fi

In [ ]:
%%capture
%%bash

# Install plink 2.0
cd /home/jupyter/
if test -e /home/jupyter/plink2; then

echo "Plink2 is already installed in /home/jupyter/"
else
echo "Plink2 is not installed"
cd /home/jupyter/

wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

unzip -o plink2_linux_x86_64_latest.zip

fi

In [ ]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink1.9

In [ ]:
%%bash

# chmod plink 2.0 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink2

In [ ]:
# Create a folder on your workspace
print("Making a working directory")
WORK_DIR = f'/home/jupyter/Subhaplotypes/'
shell_do(f'mkdir -p {WORK_DIR}') # f' stands for f-string which contains expressions inside brackets


In [ ]:
# Check directory where GP2 Tier 2 data is
print("List available imputed genotype information in GP2")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}/AAC')

#### Copy over files for the relevant ancestry:

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AAC/chr17_AAC_release7* {WORK_DIR}')

Also the eigenvec and eigenval files:

In [ ]:
# Check directory where GP2 Tier 2 data is
print("List available imputed genotype information in GP2")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_RAW_GENO_PATH}/AAC')

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_RAW_GENO_PATH}/AAC/AAC_release7.eigenval {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_RAW_GENO_PATH}/AAC/AAC_release7.eigenvec {WORK_DIR}')

#### Create a covariate file

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release7_final.csv {WORK_DIR}')

In [ ]:
clin = pd.read_csv('/home/jupyter/Subhaplotypes/master_key_release7_final.csv')
clin.info()

In [ ]:
gen = pd.read_csv('/home/jupyter/Subhaplotypes/chr17_AAC_release7.psam', sep='\t')
gen.head()

In [ ]:
pcs = pd.read_csv('/home/jupyter/Subhaplotypes/AAC_release7.eigenvec', sep='\t')
pcs.info()

In [ ]:
gen2 = pd.merge(gen, clin, left_on='#IID', right_on='GP2sampleID')
gen2.info()

In [ ]:
gen3 = pd.merge(gen2, pcs, left_on='#IID', right_on='IID')
gen3.info()

In [ ]:
plink_clin = gen3[['#IID', 'SEX', 'PHENO1', 'age_at_sample_collection', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']]
plink_clin.info()

In [ ]:
#Rename age_at_sample_collection  
plink_clin = plink_clin.rename(columns={'age_at_sample_collection': 'AGE'})
plink_clin.head()

In [ ]:
plink_clin.to_csv('/home/jupyter/Subhaplotypes/covars.txt', sep='\t', index=False, na_rep='-9',)

In [ ]:
covars = pd.read_csv('/home/jupyter/Subhaplotypes/covars.txt', sep='\t')
covars.info()

#### Remove related individuals


In [ ]:
# Select the file that matches with your population
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_META_RELEASE_PATH}/related_samples/')

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_META_RELEASE_PATH}/related_samples/AAC_release7.related {WORK_DIR}')


In [ ]:
!cat /home/jupyter/Subhaplotypes/AAC_release7.related

The IDs are:
ID1: Individual ID for the first individual of the pair
ID2: Individual ID for the second individual of the pair
We select to remove individuals in the ID1 and only exclude one person in the pair

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR


cut -d, -f2 AAC_release7.related > related_ids.txt

In [ ]:
!cat /home/jupyter/Subhaplotypes/related_ids.txt

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink2 \
--pfile chr17_AAC_release7 \
--remove related_ids.txt \
--make-pgen \
--out chr17_AAC_release7_nonrelated

#### Remove non-PD case control individuals

The prune flag keeo only these with a plink phenotype of 1 or 0

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink2 \
--pfile chr17_AAC_release7_nonrelated  \
--prune \
--make-pgen \
--out chr17_AAC_release7_nonrelated_pdc

#### Extract the region of interest (whole MAPT gene), update the variant IDs. and recode to plink v1.9 format (bed/bim/fam)

- MAPT coordinates in GRCh38: 17:45894527-46028334 (Ensembl: https://www.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000186868;r=17:45894527-46028334)

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink2 \
--pfile chr17_AAC_release7_nonrelated_pdc \
--chr 17 \
--from-bp 45894527  \
--to-bp 46028334 \
--make-pgen \
--out chr17_AAC_release7_MAPT \

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink2 \
--pfile chr17_AAC_release7_nonrelated_pdc \
--chr 17 \
--from-bp 45894554  \
--to-bp 46028334 \
--make-bed \
--out chr17_AAC_release7_MAPT \
 

To check the format of the variant IDs:

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink1.9 \
--bfile chr17_AAC_release7_MAPT \
--chr 17 \
--from-bp 45894554  \
--to-bp 46028334 \
--recode \
--out chr17_AAC_release7_MAPT \

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

head chr17_AAC_release7_MAPT.map

### Prepare file with SNPs in the subhaplotypes

In [ ]:
%%bash

# Define the working directory, adjust this as necessary
WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

# Create a file with the desired SNPs - need the coordinates
cat > snps_to_keep.txt << EOF
chr17:45908813:G:A
chr17:45942346:G:A
chr17:45977067:A:G
chr17:45998697:C:T
chr17:46003698:A:G
chr17:46028029:A:G
EOF

# Echo the contents of the file to confirm it was created correctly
cat snps_to_keep.txt


#### Extract the SNPs

- We will also add --mind to remove individuals that haven't been fully genotyped for these variants

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink1.9 \
--bfile chr17_AAC_release7_MAPT \
--extract snps_to_keep.txt \
--chr 17 \
--mind 0.01 \
--recode \
--out AAC_release7_MAPT_snps

Check the genotyping rate for the variants just to see if all variants have been genotyped properly

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink1.9 \
--file AAC_release7_MAPT_snps \
--missing \
--out AAC_release7_MAPT_snps

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

head AAC_release7_MAPT_snps.lmiss

#### Calculate HWE

.lmiss is a variant-based missing data report - F_MISS is the missing call rate and N_MISS the number of missing genotype call(s), as you can see here, we have no missing calls

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/plink2 \
--bfile chr17_AAC_release7_MAPT \
--extract snps_to_keep.txt \
--chr 17 \
--mind 0.01 \
--make-pgen \
--out AAC_release7_MAPT_snps

In [ ]:
%%bash
#We will chack if the SNPs deviate from HWE

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR

/home/jupyter/tools/plink2 \
--pfile AAC_release7_MAPT_snps \
--hardy \
--keep-if PHENO1==1 \
--out AAC_release7_MAPT_snps

In [ ]:
%%bash

WORK_DIR='/home/jupyter/Subhaplotypes/'
cd $WORK_DIR
head AAC_release7_MAPT_snps.hardy

### Run the subhaplotype analysis in R

#### Install R and packages (haplo.stat)

In [ ]:
pip install rpy2

In [ ]:
import rpy2.robjects as robjects
# Use R's install.packages function to install haplo.stats
robjects.r('install.packages("haplo.stats")')

In [ ]:
import rpy2.robjects as robjects

# Load the haplo.stats package
robjects.r('library(haplo.stats)')

In [ ]:
%load_ext rpy2.ipython

#### Run association analysis between the subhaplotypes and PD

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)
# Read the .map and .ped files generated by PLINK
map <- data.frame(fread("AAC_release7_MAPT_snps.map"))
head(map)

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)

ped <- data.frame(fread("AAC_release7_MAPT_snps.ped"))
head(ped)

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")

# rename headers in ped file to match it with the SNPs from map file, also specify alleles (a1 and a2) 
snp_identifiers <- map$V2
new_col_names <- unlist(lapply(snp_identifiers, function(snp) c(paste(snp, "a1", sep = "."), paste(snp, "a2", sep = "."))))
colnames(ped) <- c("FID", "IID", "PAT","MAT", "SEX", "PHENO")
original_snp_col_names <- colnames(ped)[7:ncol(ped)] # Adjust indices as per your data frame
names(ped)[7:ncol(ped)] <- new_col_names
head(ped)                          

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")

#Subset to keep only SNP-related columns
geno <- ped[, 7:ncol(ped)]
head(geno)


In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
#Set variables for running the association analyses in haplo.stats

#Label the SNPs:
label <- c("17:45908813","17:45942346", "17:45977067", "17:45998697","17:46003698", "17:46028029")
#Set binary pheno (0=control, 1=patient):
ped$PHENO <- ped$PHENO-1
print(ped)
y.bin <- 1*(ped$PHENO=="1")
print(y.bin)


##### Non-adjusted

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
#Non-adjusted:
H1 <- haplo.cc(y=y.bin, geno=geno, locus.label= label, control = haplo.glm.control(haplo.freq.min = 0.01, haplo.base=17))
#As we want the H2 haplotype in all populations: AGGCGG

#Sort the output on freq:
H1_ccdf <- H1$cc.df
H1_ccdf_sort <- H1_ccdf[order(-H1_ccdf$`pool.hf`),]

write.csv(H1_ccdf_sort, "Subhaplo_AAC_nonadj_r7.csv", row.names=FALSE, quote=FALSE) 
H1_ccdf_sort

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)
# Read the .map and .ped files generated by PLINK
aac <- data.frame(fread("Subhaplo_AAC_nonadj_r7.csv"))
aac

##### Adjusted

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)

adj <- data.frame(fread("covars.txt"))
head(adj)

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")

#Set binary pheno in covar (0=control, 1=patient):
adj$PHENO <- adj$PHENO1-1
adj <- data.frame(adj[,c("X.IID", "SEX","AGE","PC1", "PC2", "PC3", "PC4", "PC5")])
colnames(adj) <- c("IID","SEX", "AGE", "PC1", "PC2", "PC3", "PC4", "PC5")
nrow(adj)

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
nrow(adj)

We need to keep only those that are kept in the ped file in the covar file in order for the adjusted analyses to run:

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)

ped <- data.frame(fread("AAC_release7_MAPT_snps.ped"))
head(ped)

In [ ]:
%%R
nrow(ped)

Change age from -9 to NA in the covariate file. PLINK interpret -9 as a missing value but R does not

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
adj1 <- adj[(adj$IID %in% ped$V2),]
#adj1 <- data.frame(adj1[,c("SEX","AGE","PC1", "PC2", "PC3", "PC4", "PC5")])
head(adj1)


In [ ]:
%%R
nrow(adj1)

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
adj1[adj1 == -9] <- NA
head(adj1)

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
sum(is.na(adj1$AGE))

We also need to remove the individuals with NA in the adj1 covariate file and then remove these from the geno and y.bin for the regression

In [ ]:
%%R
adj2 <- na.omit(adj1)
nrow(adj2)

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
require(data.table)

ped <- data.frame(fread("AAC_release7_MAPT_snps.ped"))
head(ped)

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")

# rename headers in ped file to match it with the SNPs from map file, also specify alleles (a1 and a2) 
snp_identifiers <- map$V2
new_col_names <- unlist(lapply(snp_identifiers, function(snp) c(paste(snp, "a1", sep = "."), paste(snp, "a2", sep = "."))))
colnames(ped) <- c("FID", "IID", "PAT","MAT", "SEX", "PHENO")
original_snp_col_names <- colnames(ped)[7:ncol(ped)] # Adjust indices as per your data frame
names(ped)[7:ncol(ped)] <- new_col_names
head(ped) 

Remove individuals with missing covariate data

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")
ped <- ped[(ped$IID %in% adj2$IID),]
nrow(ped)

In [ ]:
%%R
# Set the working directory to the Terra platform directory
setwd("/home/jupyter/Subhaplotypes/")

#Subset to keep only SNP-related columns
geno <- ped[, 7:ncol(ped)]
nrow(geno)


In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
#Set variables for running the association analyses in haplo.stats

#Label the SNPs:
label <- c("17:45908813","17:45942346", "17:45977067", "17:45998697","17:46003698", "17:46028029")
#Set binary pheno (0=control, 1=patient):
ped$PHENO <- ped$PHENO-1
head(ped)
y.bin <- 1*(ped$PHENO=="1")

In [ ]:
%%R
adj2 <- data.frame(adj2[,c("SEX","AGE","PC1", "PC2", "PC3", "PC4", "PC5")])
head(adj2)

In [ ]:
%%R
adj2 <- data.matrix(adj2)

Run the association analyses between the subhaplotypes and PD adjusted by covariates

In [ ]:
%%R
setwd("/home/jupyter/Subhaplotypes/")
#Adjusted for age, sex, PC1-5:
require(rms)

H1_adj <- haplo.cc(y=y.bin, geno=geno, locus.label=label, x.adj=adj2, control = haplo.glm.control(haplo.freq.min = 0.01, haplo.base=17))
H1_adj

#Sort the output on p-value and save:
H1_adj_ccdf <- H1_adj$cc.df
H1_adj_ccdf_sort <- H1_adj_ccdf[order(-H1_adj_ccdf$`pool.hf`),]
write.csv(H1_adj_ccdf_sort, "Subhaplo_AAC_adj_r7.csv", row.names=FALSE, quote=FALSE) 
head(H1_adj_ccdf_sort)

### Save output


In [ ]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}Subhaplo_AAC_nonadj_r7.csv {WORKSPACE_BUCKET}')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}Subhaplo_AAC_adj_r7.csv {WORKSPACE_BUCKET}')